# Seperating stock to forcast for certain dates depending on the product

In [1]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn statsmodels


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split

In [3]:
import sys
sys.path.append('../../src')

In [4]:
from data_config import ProductHandler as ph
from app_utils import Transform as trans
from model_utils import *
from sklearn.pipeline import Pipeline

/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:13: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:15: DtypeWarning: Columns (5,29,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:18: DtypeWarning: Columns (43,57,106,141,146,211,212,247,255,294,298,316,327,332,350,354) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  sp_inv_adds = pd.read_csv(


----- DROP BOOLEAN AND NULL COLUMNS (NOT NEEDED) -----

Merged Data - Columns to be dropped (Missing Vals): ['BackStage_DefaultReportForm', 'BackStage_SmartMarkSet', 'BillCounty', 'CoreType', 'EndUserNum', 'EndUserPO', 'Equip3_ID', 'Equip3_Status', 'Est_v_Act_Notes', 'Ink_Status', 'JDF_Note_to_DFE', 'JDF_Send_Msg', 'MiscChargeDesc1', 'MiscChargeDesc2', 'MiscChargeDesc3', 'MiscChargeDesc4', 'PlateStat', 'ProofStat', 'Roto_CEL_Product_ID', 'Roto_Quote_Line_ID', 'Roto_Quote_Number', 'Schedule_Status', 'ShipAttn_EmailAddress', 'ShipCounty', 'ShipStat', 'Tag_x', 'Tool3Descr', 'Tool4Descr', 'Tool5Descr', 'ToolNo3', 'ToolNo4', 'ToolNo5', 'ToolStat', 'UserDef_MR_1_Lb', 'UserDef_MR_2_Lb', 'Assigned', 'Desc2_x', 'ediLineNumber', 'eTraxx_Customer_Notes', 'Location_x', 'PO_Number', 'Adhesive', 'Alternate', 'eTraxx_Forecast_Range', 'Material', 'Tag_y', 'UPC']

Merged Data - Columns to be dropped (All cols with rows same vals): ['Act_OTHER_Hours', 'ActArtwork', 'ActFinMaterial', 'ActPostPressHours',

/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/sarimax/../../src/datasets.py:110: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-e

In [5]:
ph.custom_prod_set()

All custom codes for existing products: 
{'066', '531', '89X', '15S', 'PFG', '27S', '345', 'LC-', '755', 'TMC', '88X', '739', '150', '02X', '057', '10Y', 'CHW', 'LS-', '447', '727', 'CLD', '621', '779', '786', 'HB-', 'EN-', '23P', 'HN-', '94W', '685', 'SM-', '525', '72W', '624', '122', 'I-T', 'MX-', '942', '58U', 'EM-', '119', '19Z', '07Q', '436', '580', '897', '54X', '452', '898', '13Z', '704', '914', '99Z', 'SFR', '80X', '79Q', '753', 'KIL', '74Z', '1Y-', '348', 'I-0', '024', 'HAH', '78Y', '330', '03T', '265', 'AND', '760', '313', '725', '10X', '23Q', '471', 'BRO', '746', '152', '360', '00-', '340', '37Y', '40T', '14U', 'JBS', 'LZ-', '850', '6-1', 'SOU', '560', '62S', 'MAS', 'RLC', 'LBL', '626', 'RS-', '718', '260', '07-', '660', 'HAN', '88Z', '266', '27Y', '04U', '557', '100', '71Y', 'TK-', '351', '06X', '195', '480', '165', '91U', '42U', 'ILS', '113', 'LAT', '158', 'FLO', '40S', '26X', '226', 'GLO', 'ELS', '247', '561', 'XP-', '48U', '12Z', '867', '783', '305', 'SS-', '46Z', 'U-0',

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# define the features and target variable from 'product_sales'
# CHANGE THE CUSTOMER CODE HERE >>>>>>>>>>
custom_code_df = pd.DataFrame(ph.get_custom_code_data('FRE'))

# customer code for later referencing
custom_ref = custom_code_df.loc[0, 'ProductNumber'][:3].lower().upper()
print(custom_ref)

# Set OrderDate as the index (this is necessary for time series models)
custom_code_df.set_index('OrderDate', inplace=True)

custom_code_df.head()

FRE


,ProductNumber,order_year,order_month,order_week,order_weekday,is_weekend,Customer_Num,OrderQuantity,prev_month_sales,prev_week_sales,...,moving_avg_3m,moving_avg_6m,moving_avg_12m,moving_avg_18m,sales_2022,sales_2023,sales_2024,growth_2023,growth_2024,PhysicalInv
OrderDate,,,,,,,,,,,,,,,,,,,,,
2022-01-18,FRE-005-0006Y,2022,1,3,1,0,66.0,40000,80000.00000,80000.00000,...,53333.333333,61000.000000,61000.000000,61000.000000,40000,0,0,-100.0,NaN,0
2022-03-09,FRE-005-0006Y,2022,3,10,2,0,66.0,80000,40000.00000,40000.00000,...,50000.000000,66250.000000,66250.000000,66250.000000,80000,0,0,-100.0,NaN,0
2022-04-07,FRE-005-0006Y,2022,4,14,3,0,66.0,40000,30000.00000,30000.00000,...,61666.666667,61666.666667,61666.666667,61666.666667,40000,0,0,-100.0,NaN,0
2022-06-14,FRE-005-0006Y,2022,6,24,1,0,66.0,30000,115000.00000,115000.00000,...,72500.000000,72500.000000,72500.000000,72500.000000,30000,0,0,-100.0,NaN,0
2022-07-26,FRE-005-0006Y,2022,7,30,1,0,66.0,115000,110031.53337,110031.53337,...,115000.000000,115000.000000,115000.000000,115000.000000,115000,0,0,-100.0,NaN,0


In [7]:
# Compute OrderQuantity values with a zscore with threshold of 3 and remove them (Outlier removal)
custom_code_df = trans.compute_zscore(custom_code_df)

# Convert ProductNumber to categorical
custom_code_df['ProductNumber'] = custom_code_df['ProductNumber'].astype('category')
custom_code_df['ProductNumber_code'] = custom_code_df['ProductNumber'].cat.codes

custom_code_df.head()

ProductNumber  order_year  order_month  order_week  \
ProductNumber OrderDate                                                        
FRE-005-0006Y 2022-01-18  FRE-005-0006Y        2022            1           3   
              2022-03-09  FRE-005-0006Y        2022            3          10   
              2022-04-07  FRE-005-0006Y        2022            4          14   
              2022-06-14  FRE-005-0006Y        2022            6          24   
              2022-07-26  FRE-005-0006Y        2022            7          30   

                          order_weekday  is_weekend  Customer_Num  \
ProductNumber OrderDate                                             
FRE-005-0006Y 2022-01-18              1           0          66.0   
              2022-03-09              2           0          66.0   
              2022-04-07              3           0          66.0   
              2022-06-14              1           0          66.0   
              2022-07-26              1           0          66.0   

                          OrderQuantity  prev_month_sales  prev_week_sales  \
ProductNumber OrderDate                                                      
FRE-005-0006Y 2022-01-18          40000       80000.00000      80000.00000   
              2022-03-09          80000       40000.00000      40000.00000   
              2022-04-07          40000       30000.00000      30000.00000   
              2022-06-14          30000      115000.00000     115000.00000   
              2022-07-26         115000      110031.53337     110031.53337   

                          ...  moving_avg_12m  moving_avg_18m  sales_2022  \
ProductNumber OrderDate   ...                                               
FRE-005-0006Y 2022-01-18  ...    61000.000000    61000.000000       40000   
              2022-03-09  ...    66250.000000    66250.000000       80000   
              2022-04-07  ...    61666.666667    61666.666667       40000   
              2022-06-14  ...    72500.000000    72500.000000       30000   
              2022-07-26  ...   115000.000000   115000.000000      115000   

                          sales_2023  sales_2024  growth_2023  growth_2024  \
ProductNumber OrderDate                                                      
FRE-005-0006Y 2022-01-18           0           0       -100.0          NaN   
              2022-03-09           0           0       -100.0          NaN   
              2022-04-07           0           0       -100.0          NaN   
              2022-06-14           0           0       -100.0          NaN   
              2022-07-26           0           0       -100.0          NaN   

                          PhysicalInv  z_score  ProductNumber_code  
ProductNumber OrderDate                                             
FRE-005-0006Y 2022-01-18            0  0.65625                   0  
              2022-03-09            0  0.59375                   0  
              2022-04-07            0  0.65625                   0  
              2022-06-14            0  0.96875                   0  
              2022-07-26            0  1.68750                   0  

[5 rows x 35 columns]

In [8]:
X = custom_code_df.drop('OrderQuantity', axis=1)
y = custom_code_df.OrderQuantity